# Загрузка и обработка данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def parse_tskv_line(line):
    # Разделяем строку по табуляции и создаём словарь из пар ключ-значение.
    return dict(item.split('=', 1) for item in line.strip().split('\t'))

# Чтение файла строка за строкой и преобразование каждой строки через parse_tskv_line
with open('/kaggle/input/yandex-data/geo-reviews-dataset-2023.tskv', 'r', encoding='utf-8') as file:
    data = [parse_tskv_line(line) for line in file]

# Создание DataFrame из списка словарей
df = pd.DataFrame(data)

# Вывод первых нескольких строк DataFrame
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [ ]:
# Убираем пустые значения оценок
df = df[df['rating'] != '0.']

# Делаем классы от 0 до 4 для удобства обучения
df["label"] = df.rating.apply(lambda x: int(x[0]) - 1)

# Поменяли имя колонки с 'rating' на 'label'
df = df.drop(columns=['rating'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499800 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  499800 non-null  object
 1   name_ru  498830 non-null  object
 2   rubrics  499800 non-null  object
 3   text     499800 non-null  object
 4   label    499800 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 22.9+ MB


In [ ]:
df['text'] = df['text'].apply(lambda x: x.replace('\\n', ''))
df.head()

,address,name_ru,rubrics,text,label
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,Жилой комплекс,Московский квартал 2.Шумно : летом по ночам ди...,2
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...",4
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...",0
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал.Н...,3
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,4


In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

In [ ]:
#Выполняем балансировку классов
import pandas as pd
import random

counts = df['label'].value_counts()
min_count = counts.min()

df_balanced = pd.DataFrame()

for label in counts.index:
    group = df[df['label'] == label]

    if len(group) > min_count:
        group = group.sample(n=min_count)

    df_balanced = pd.concat([df_balanced, group])



In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_balanced[['text', 'label']], test_size=0.3, random_state=42)

In [ ]:
with open('/kaggle/input/yandex-data/test.tskv', 'r', encoding='utf-8') as file:
    data = [parse_tskv_line(line) for line in file]

df_test = pd.DataFrame(data)
df_test.head()

,address,name_ru,rubrics,text
0,"Московская обл., Электроугли, ул. Школьная, вб...",Школьная,Жилой комплекс,Как то он выгледит по другому и не так красиво...
1,"Москва, Ленинский проспект, 34/1",Банк ВТБ,Банк,Не решили вопрос \nВ приложении втб прислали с...
2,"Москва, улица Плеханова, 17, стр. 6",Перово Плаза,Гостиница,"Ужасное место,при заезде не могли никак засели..."
3,"Москва, Каширское шоссе, 57, корп. 4",Пятёрочка,Супермаркет,Всегда нереально медленные кассиры
4,"Москва, Головинское шоссе, 5, корп. 1",О'кей,Продуктовый гипермаркет;Гипермаркет,Самое ужасное мясо на развес. Всегда продают т...


In [ ]:
df_test['text'] = df_test['text'].apply(lambda x: x.replace('\\n', ''))

In [ ]:
train = Dataset.from_pandas(train_df)
val = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(pd.DataFrame(df_test['text']))

ds = DatasetDict()

ds['train'] = train
ds['validation'] = val
ds['test'] = test

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)


tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/42308 [00:00<?, ? examples/s]

Map:   0%|          | 0/18132 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

# **Подготавливаем модель для обучения**

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=5)

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy", "f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import torch

torch.cuda.current_device()

0

In [ ]:
!pip install peft

In [ ]:
#Lora config можно использовать для LLM, уже обученной на похожей задаче, для ускорения обучения
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS,
                         inference_mode=False,
                         r=8,
                         lora_alpha=32,
                         lora_dropout=0.1,
                         use_rslora=True,
                         init_lora_weights="pissa",
                         )
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  learning_rate=2e-5,
	                              num_train_epochs=10,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  gradient_accumulation_steps=4,
                                  fp16=True,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  weight_decay=0.01,
                                  )

In [ ]:
training_args.device

device(type='cuda', index=0)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,)

In [ ]:
trainer.train()

# Загрузка тестовых данных и разметка

In [ ]:
df_test.info()

In [ ]:
import numpy as np

predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
preds = [str(x + 1) + "." for x in preds]

df_test['rating'] = preds
df_test.head()

In [ ]:
with open('test_with_rating.tskv', 'w') as file:
    for index, row in df_test.iterrows():
        line = ''
        for column in df_test.columns:
            line += f'{column}={row[column]}\t'
        print(line.strip(), end='\n', file=file)

In [ ]:
with open('test_with_rating.tskv', 'r') as file:
    data = [parse_tskv_line(line) for line in file]

df_test = pd.DataFrame(data)
df_test.head()

In [ ]:
df_test.info()